Merging the xT, VAEP and Expected VAEP across chains.

In [1]:
import pandas as pd
import numpy as np
import joblib

import sys
sys.path.append("..")
sys.path.append("/Users/ciaran/Documents/Projects/AFL/git-repositories/afl-player-ratings/expected-threat")
sys.path.append("/Users/ciaran/Documents/Projects/AFL/git-repositories/afl-player-ratings/vaep")
sys.path.append("/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/")
sys.path.append("/Users/ciaran/Documents/Projects/AFL/git-repositories/afl-player-ratings/exp-vaep")


import vaep.domain.preprocessing.preprocessing as vaep
import vaep.domain.preprocessing.formula as vaep_formula
import exp_vaep.domain.preprocessing.preprocessing as exp_vaep
import exp_vaep.domain.preprocessing.formula as exp_vaep_formula
from score_chain_data import *

import os
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

In [2]:
chains = pd.read_csv(chain_file_path)
chains.head()

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Season
0,1,centreBounce,goal,1.0,1,13,13.0,Brisbane Lions,NaN,NaN,NaN,Centre Bounce,0.0,0.0,NaN,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN
1,1,centreBounce,goal,2.0,1,13,24.0,Brisbane Lions,Brisbane Lions,Dayne Zorko,Dayne_Zorko,Hard Ball Get,8.0,-5.0,NaN,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN
2,1,centreBounce,goal,3.0,1,13,24.0,Brisbane Lions,Brisbane Lions,Dayne Zorko,Dayne_Zorko,Handball,9.0,-6.0,ineffective,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN
3,1,centreBounce,goal,4.0,1,13,28.0,Brisbane Lions,Sydney,Oliver Florent,Oliver_Florent,Loose Ball Get,11.0,-7.0,NaN,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN
4,1,centreBounce,goal,5.0,1,13,29.0,Brisbane Lions,Sydney,Oliver Florent,Oliver_Florent,Handball,12.0,-5.0,effective,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN


Get Expected Threat

In [3]:
chains = score_xTModel(chains)
chains.head()

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Season,ReceiverTeam,Receiver,type_name,outcome_name,shoot,move,goal,end_x,end_y,xT_created,xT_denied
0,1,centreBounce,goal,1.0,1,13,13.0,Brisbane Lions,NaN,NaN,NaN,Centre Bounce,0.0,0.0,NaN,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,centreBounce,goal,2.0,1,13,24.0,Brisbane Lions,Brisbane Lions,Dayne Zorko,Dayne_Zorko,Hard Ball Get,8.0,-5.0,NaN,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN,Brisbane Lions,Dayne Zorko,Carry,True,False,True,False,9.0,-6.0,0.005595,NaN
2,1,centreBounce,goal,3.0,1,13,24.0,Brisbane Lions,Brisbane Lions,Dayne Zorko,Dayne_Zorko,Handball,9.0,-6.0,ineffective,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN,Sydney,Oliver Florent,Handball,False,False,True,False,11.0,-7.0,NaN,0.0
3,1,centreBounce,goal,4.0,1,13,28.0,Brisbane Lions,Sydney,Oliver Florent,Oliver_Florent,Loose Ball Get,11.0,-7.0,NaN,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN,Sydney,Oliver Florent,Carry,True,False,True,False,12.0,-5.0,0.000000,NaN
4,1,centreBounce,goal,5.0,1,13,29.0,Brisbane Lions,Sydney,Oliver Florent,Oliver_Florent,Handball,12.0,-5.0,effective,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN,Sydney,George Hewett,Handball,True,False,True,False,14.0,-2.0,0.007561,NaN


Score VAEP

In [ ]:
schema_chains = vaep.convert_chains_to_schema(chains)
gamestate_features = vaep.create_gamestate_features(schema_chains)

In [ ]:
vaep_modelling_data = pd.concat([schema_chains, gamestate_features], axis=1)

Load models

In [ ]:
model_file_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/afl-player-ratings/vaep/model_outputs/models/"
scoring_model = "vaep_scoring_v1.joblib"
conceding_model = "vaep_conceding_v1.joblib"

In [ ]:
score_model = joblib.load(model_file_path + "/" + scoring_model)
concede_model = joblib.load(model_file_path + "/" + conceding_model)

Make predictions

In [ ]:
model_features = score_model.xgb_model.get_booster().feature_names

In [ ]:
vaep_features = vaep_modelling_data[model_features]

In [ ]:
schema_chains['scores'] = score_model.predict_proba(vaep_features, calibrate=True)
schema_chains['concedes'] = concede_model.predict_proba(vaep_features, calibrate=True)

Compute VAEP

In [ ]:
match_list = list(schema_chains['match_id'].unique())
match_vaep_list = []
for match in match_list:
    match_chains = schema_chains[schema_chains['match_id'] == match]
    v = vaep_formula.value(match_chains, match_chains['scores'], match_chains['concedes'])
    match_vaep_list.append(v)
    
vaep_values = pd.concat(match_vaep_list, axis=0)

In [ ]:
chain_vaep_data = pd.concat([schema_chains, vaep_values], axis=1)
chain_vaep_data.head()

Score Expected VAEP

Get Expected Scores

In [ ]:
expected_score_version = 4
model_location = '/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/models/'
expected_goal_set_version = 7
expected_behind_set_version = 5
expected_miss_set_version = 4

expected_goal_open_version = 7
expected_behind_open_version = 6
expected_miss_open_version = 5

In [ ]:
expected_goal_set_model = joblib.load(model_location+"expected_goal_set_v"+str(expected_goal_set_version)+".joblib")
expected_behind_set_model = joblib.load(model_location+"expected_behind_set_v"+str(expected_behind_set_version)+".joblib")
expected_miss_set_model = joblib.load(model_location+"expected_miss_set_v"+str(expected_miss_set_version)+".joblib")

expected_goal_open_model = joblib.load(model_location+"expected_goal_open_v"+str(expected_goal_open_version)+".joblib")
expected_behind_open_model = joblib.load(model_location+"expected_behind_open_v"+str(expected_behind_open_version)+".joblib")
expected_miss_open_model = joblib.load(model_location+"expected_miss_open_v"+str(expected_miss_open_version)+".joblib")

In [ ]:
goal_set_features = expected_goal_set_model.xgb_model.get_booster().feature_names
behind_set_features = expected_behind_set_model.xgb_model.get_booster().feature_names
miss_set_features = expected_miss_set_model.xgb_model.get_booster().feature_names

goal_open_features = expected_goal_open_model.xgb_model.get_booster().feature_names
behind_open_features = expected_behind_open_model.xgb_model.get_booster().feature_names
miss_open_features = expected_miss_open_model.xgb_model.get_booster().feature_names

In [ ]:
chains[['ballUp', 'centreBounce', 'kickIn', 'possGain', 'throwIn']] = pd.get_dummies(chains['Initial_State'])

In [ ]:
chains['Event_Type1'] = chains['Description'].shift(1)
shots = chains[chains['Shot_At_Goal'] == True]
shots['Set_Shot'] = shots['Event_Type1'].apply(lambda x: ("Mark" in x) or ("Free" in x))
set_shots = shots[shots['Set_Shot']]
open_shots = shots[~shots['Set_Shot']]

In [ ]:
set_shots = exp_vaep.expected_score_feature_engineering(set_shots)
open_shots = exp_vaep.expected_score_feature_engineering(open_shots)

In [ ]:
set_shots['xGoals'] = expected_goal_set_model.predict_proba(set_shots[goal_set_features], calibrate=True)
set_shots['xBehinds'] = expected_behind_set_model.predict_proba(set_shots[behind_set_features], calibrate=True)
set_shots['xMiss'] = expected_miss_set_model.predict_proba(set_shots[miss_set_features], calibrate=True)

open_shots['xGoals'] = expected_goal_open_model.predict_proba(open_shots[goal_open_features], calibrate=True)
open_shots['xBehinds'] = expected_behind_open_model.predict_proba(open_shots[behind_open_features], calibrate=True)
open_shots['xMiss'] = expected_miss_open_model.predict_proba(open_shots[miss_open_features], calibrate=True)

In [ ]:
exp_shots = pd.concat([set_shots, open_shots], axis=0)
exp_shots = exp_shots.sort_values(by = ['Match_ID', "Chain_Number", "Order"])
shots = shots.merge(exp_shots[['Chain_Number', 'Order', 'Match_ID', 'xGoals', 'xBehinds', 'xMiss']], how = "left", on = ['Chain_Number', 'Order', 'Match_ID'])

In [ ]:
shots['xGoals_normalised'] = shots['xGoals'] / (shots['xGoals'] + shots['xBehinds'] + shots['xMiss'])
shots['xBehinds_normalised'] = shots['xBehinds'] / (shots['xGoals'] + shots['xBehinds'] + shots['xMiss'])
shots['xMiss_normalised'] = shots['xMiss'] / (shots['xGoals'] + shots['xBehinds'] + shots['xMiss'])
shots['xScore'] = shots['xGoals_normalised']*6 + shots['xBehinds_normalised']

In [ ]:
chains = chains.merge(shots[['Match_ID', 'Chain_Number', 'Order', 'xScore', 'xGoals_normalised']], how="left", on = ['Match_ID', 'Chain_Number', 'Order'])
chains[['xScore', 'xGoals_normalised']] = chains[['xScore', 'xGoals_normalised']].fillna(0)
assert chains.shape[0] == number_of_rows
chains.head()

Preprocess for Expected VAEP

In [ ]:
schema_chains = exp_vaep.convert_chains_to_schema(chains)
gamestate_features = exp_vaep.create_gamestate_features(schema_chains)

In [ ]:
schema_chains.head()

In [ ]:
exp_vaep_modelling_data = pd.concat([schema_chains, gamestate_features], axis=1)

Load models

In [ ]:
model_file_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/afl-player-ratings/exp_vaep/model_outputs/models/"
scoring_model = "exp_vaep_scoring_v1.joblib"
conceding_model = "exp_vaep_conceding_v1.joblib"

In [ ]:
exp_score_model = joblib.load(model_file_path + "/" + scoring_model)
exp_concede_model = joblib.load(model_file_path + "/" + conceding_model)

In [ ]:
exp_model_features = exp_score_model.get_booster().feature_names

In [ ]:
exp_vaep_features = exp_vaep_modelling_data[exp_model_features]

Make predictions

In [ ]:
schema_chains['scores'] = np.clip(exp_score_model.predict(exp_vaep_features), 0, 6)
schema_chains['concedes'] = np.clip(exp_concede_model.predict(exp_vaep_features), 0, 6)

In [ ]:
schema_chains['scores'].describe()

Compute VAEP

In [ ]:
match_list = list(schema_chains['match_id'].unique())
match_exp_vaep_list = []
for match in match_list:
    match_chains = schema_chains[schema_chains['match_id'] == match]
    v = exp_vaep_formula.value(match_chains, match_chains['scores'], match_chains['concedes'])
    match_exp_vaep_list.append(v)
    
exp_vaep_values = pd.concat(match_exp_vaep_list, axis=0)

In [ ]:
chain_exp_vaep_data = pd.concat([schema_chains, exp_vaep_values], axis=1)
chain_exp_vaep_data.head(12)

In [ ]:
chain_exp_vaep_data = chain_exp_vaep_data.rename(columns = {
    "scores":"exp_scores",
    "concedes":"exp_concedes",
    "offensive_value":"exp_offensive_value",
    "defensive_value":"exp_defensive_value",
    "vaep_value":"exp_vaep_value"})
chain_exp_vaep_data.head()

In [ ]:
chain_vaep_data = chain_vaep_data.merge(chain_exp_vaep_data[['match_id', 'chain_number', 'order', 'exp_scores', 'exp_concedes', 'exp_offensive_value', 'exp_defensive_value', "exp_vaep_value"]], how = "left", on=['match_id', 'chain_number', 'order'])
chain_vaep_data.head()

In [ ]:
chains = chains.merge(chain_vaep_data[['match_id', 'chain_number', 'order', "overall_seconds", 'action_type', 'outcome_type', 'scores', 'concedes', 'offensive_value', 'defensive_value', "vaep_value", 'exp_scores', 'exp_concedes', 'exp_offensive_value', 'exp_defensive_value', "exp_vaep_value"]], how = "left", left_on = ['Match_ID', 'Chain_Number', 'Order'], right_on=['match_id', 'chain_number', 'order'])
assert chains.shape[0] == number_of_rows
chains.head()

Export

In [ ]:
chains.to_csv("/Users/ciaran/Documents/Projects/AFL/git-repositories/afl-player-ratings/data/chains_xt_vaep.csv", index=False)